# Proyect Wyscout


In [1]:
import pandas as pd
import glob
import os
import unicodedata
import re

pd.options.display.max_columns = None

## Teams


In [47]:
# Ruta relativa desde la carpeta scripts
# folder_path = '../data/input/Betplay/'
folder_path = '../data/input/LigaEcu/teams/'
folder_path = '../data/input/Argentina_B/teams/'

# Obtener lista de archivos Excel
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Leer todos los archivos
dfs = [pd.read_excel(file) for file in excel_files]

# Concatenar todos los DataFrames en uno
combined_df = pd.concat(dfs, ignore_index=True)

# Ver ejemplo
print(f'{len(dfs)} archivos leídos')

18 archivos leídos


In [48]:
# Ruta relativa desde la carpeta scripts
# folder_path = '../data/input/LigaEcu/teams/'
# folder_path = '../data/input/Betplay/'
folder_path = '../data/input/Argentina_B/teams/'


# Obtener lista de archivos Excel
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Leer todos los archivos y almacenarlos en una lista
dfs = []
for file in excel_files:
    df = pd.read_excel(file)
    print(f"Archivo: {os.path.basename(file)} - Número de columnas: {df.shape[1]}")
    dfs.append(df)

# Concatenar todos los DataFrames en uno
combined_df = pd.concat(dfs, ignore_index=True)

# Ver ejemplo
print(f'{len(dfs)} archivos leídos')


Archivo: Team Stats Agropecuario.xlsx - Número de columnas: 109
Archivo: Team Stats Almirante Brown.xlsx - Número de columnas: 109
Archivo: Team Stats Central Norte.xlsx - Número de columnas: 109
Archivo: Team Stats Chacarita Juniors.xlsx - Número de columnas: 109
Archivo: Team Stats Chaco For Ever.xlsx - Número de columnas: 109
Archivo: Team Stats Club Atlético Mitre.xlsx - Número de columnas: 109
Archivo: Team Stats Colón.xlsx - Número de columnas: 109
Archivo: Team Stats Defensores de Belgrano.xlsx - Número de columnas: 109
Archivo: Team Stats Defensores Unidos.xlsx - Número de columnas: 109
Archivo: Team Stats Deportivo Morón.xlsx - Número de columnas: 109
Archivo: Team Stats Estudiantes Caseros.xlsx - Número de columnas: 109
Archivo: Team Stats Estudiantes Río Cuarto.xlsx - Número de columnas: 109
Archivo: Team Stats Gimnasia Jujuy.xlsx - Número de columnas: 109
Archivo: Team Stats Gimnasia Mendoza.xlsx - Número de columnas: 109
Archivo: Team Stats Nueva Chicago.xlsx - Número 

In [53]:
combined_df.head(5)
combined_df['Competición'].value_counts()

liga = combined_df['Competición'].unique()[1]
print(f'Competición: {liga}')

Competición: Argentina. Primera Nacional


In [54]:
df = combined_df.copy()

df = df.dropna(subset=['Partido'])

def rename_columns_with_slash(columns):
    new_columns = []
    i = 0
    while i < len(columns):
        col = columns[i]

        # Si tiene "/", expandimos usando el número de partes
        if "/" in col:
            parts = [p.strip() for p in col.split("/")]
            base = parts[0]
            for j in range(len(parts)):
                if i + j < len(columns):
                    name = base if j == 0 else f"{base} {parts[j]}"
                    new_columns.append(name)
            i += len(parts)
        # Si es 'Unnamed', intenta continuar desde antes
        elif "Unnamed" in col:
            # Si no hemos expandido antes, darle nombre genérico
            new_columns.append(f"%_{parts[0]}")
            i += 1
        else:
            new_columns.append(col)
            i += 1

    # Rellenar si quedaron columnas faltantes
    while len(new_columns) < len(columns):
        new_columns.append(f"col_{len(new_columns)}")

    return new_columns

# Aplicar al DataFrame
df.columns = rename_columns_with_slash(list(df.columns))

df['Fecha'].value_counts().sort_index(ascending=False)
df['Competición'].value_counts().sort_index(ascending=False)

# df = df[df['Competición'] == 'Colombia. Liga BetPlay']
df = df[df['Competición'] == liga]

df = df[df['Fecha'] >= '2025-01-01']

df.drop_duplicates(inplace=True)

print(df.shape)

df.head(2)

(372, 109)


,Fecha,Partido,Competición,Duración,Equipo,Seleccionar esquema,Goles,xG,Tiros,Tiros a la portería,%_Tiros,Pases,Pases logrados,%_Pases,"Posesión del balón, %",Balones perdidos,Balones perdidos bajos,Balones perdidos medios,Balones perdidos altos,Balones recuperados,Balones recuperados bajos,Balones recuperados medios,Balones recuperados altos,Duelos,Duelos ganados,%_Duelos,Tiros de fuera del área,Tiros de fuera del área a la portería,%_Tiros de fuera del área,Ataques posicionales,Ataques posicionales con remate,%_Ataques posicionales,Contraataques,Contraataques con remate,%_Contraataques,Jugadas a balón parado,Jugadas a balón parado con remate,%_Jugadas a balón parado,Córneres,Córneres con remate,%_Córneres,Tiros libres,Tiros libres con remate,%_Tiros libres,Penaltis,Penaltis marcados,%_Penaltis,Centros,Centros precisos,%_Centros,Pases cruzados en profundidad completados,Pases en profundidad completados,Entradas al área de penalti (carreras,Entradas al área de penalti (carreras pases cruzados),%_Entradas al área de penalti (carreras,Toques en el área de penalti,Duelos ofensivos,Duelos ofensivos ganados,%_Duelos ofensivos,Fuera de juego,Goles recibidos,Tiros en contra,Tiros en contra a la portería,%_Tiros en contra,Duelos defensivos,Duelos defensivos ganados,%_Duelos defensivos,Duelos aéreos,Duelos aéreos ganados,%_Duelos aéreos,Entradas a ras de suelo,Entradas a ras de suelo logradas,%_Entradas a ras de suelo,Interceptaciones,Despejes,Faltas,Tarjetas amarillas,Tarjetas rojas,Pases hacia adelante,Pases hacia adelante logrados,%_Pases hacia adelante,Pases hacia atrás,Pases hacia atrás logrados,%_Pases hacia atrás,Pases laterales,Pases laterales logrados,%_Pases laterales,Pases largos,Pases largos logrados,%_Pases largos,Pases en el último tercio,Pases en el último tercio logrados,%_Pases en el último tercio,Pases progresivos,Pases progresivos precisos,%_Pases progresivos,Desmarques,Desmarques logrados,%_Desmarques,Saques laterales,Saques laterales logrados,%_Saques laterales,Saques de meta,Intensidad de paso,Promedio pases por posesión del balón,Lanzamiento largo %,Distancia media de tiro,Longitud media pases,PPDA
2,2025-07-06,Agropecuario - Talleres Remedios 2:1,Argentina. Primera Nacional,96.0,Agropecuario,3-4-3 (100.0%),2.0,1.89,15.0,7.0,46.67,491.0,399.0,81.26,63.07,125.0,12.0,30.0,83.0,108.0,39.0,49.0,20.0,220.0,105.0,47.73,3.0,1.0,33.33,42.0,9.0,21.43,1.0,1.0,100.0,28.0,4.0,14.29,12.0,3.0,25.00,3.0,0.0,0.0,0.0,0.0,0.0,33.0,8.0,24.24,8.0,7.0,42.0,6.0,22.0,20.0,99.0,40.0,40.40,1.0,1.0,4.0,1.0,25.00,63.0,40.0,63.49,32.0,14.0,43.75,4.0,1.0,25.00,31.0,14.0,15.0,2.0,0.0,175.0,133.0,76.00,71.0,68.0,95.77,185.0,153.0,82.70,74.0,46.0,62.16,90.0,65.0,72.22,92.0,77.0,83.70,0.0,0.0,0.0,18.0,18.0,100.00,3.0,15.28,4.2,15.07,14.74,20.77,5.86
3,2025-07-06,Agropecuario - Talleres Remedios 2:1,Argentina. Primera Nacional,96.0,Talleres Remedios,4-2-3-1 (48.89%),1.0,0.56,4.0,1.0,25.00,283.0,192.0,67.84,36.93,138.0,37.0,55.0,46.0,86.0,54.0,25.0,7.0,220.0,110.0,50.00,1.0,0.0,0.00,17.0,2.0,11.76,0.0,0.0,0.0,26.0,2.0,7.69,3.0,1.0,33.33,2.0,0.0,0.0,0.0,0.0,0.0,18.0,8.0,44.44,6.0,1.0,18.0,1.0,14.0,8.0,63.0,23.0,36.51,0.0,2.0,15.0,8.0,53.33,99.0,59.0,59.60,32.0,16.0,50.00,12.0,4.0,33.33,63.0,29.0,13.0,3.0,0.0,132.0,80.0,60.61,32.0,27.0,84.38,90.0,65.0,72.22,54.0,24.0,44.44,41.0,22.0,53.66,66.0,35.0,53.03,0.0,0.0,0.0,17.0,11.0,64.71,10.0,15.03,2.6,19.08,16.07,21.42,11.71


In [58]:
df['Equipo'].unique()

array(['Agropecuario', 'Talleres Remedios', 'Estudiantes Río Cuarto',
       'Defensores de Belgrano', 'Chacarita Juniors', 'San Telmo',
       'Nueva Chicago', 'Gimnasia Jujuy', 'Gimnasia Mendoza',
       'Central Norte', 'Almirante Brown', 'Chaco For Ever',
       'Club Atlético Mitre', 'Colón', 'Temperley', 'Defensores Unidos',
       'Estudiantes Caseros', 'Deportivo Morón'], dtype=object)

In [55]:
# 1. Columnas identificadoras y métricas
id_cols = ['Fecha', 'Partido', 'Equipo']
metric_cols = [col for col in df.columns if col not in id_cols]

# 2. Preparar el DataFrame contrario
df_contra = df.copy()
df_contra = df_contra.rename(
    columns={**{col: f"{col}_contra" for col in metric_cols}, 'Equipo': 'Equipo_contra'}
)

# 3. Hacer merge por Fecha y Partido (equipo contrario)
df_full = df.merge(
    df_contra,
    on=['Fecha', 'Partido'],
    suffixes=('', '_drop')
)

# 4. Filtrar para evitar unir el equipo consigo mismo
df_full = df_full[df_full['Equipo'] != df_full['Equipo_contra']]

# 5. Eliminar columnas "_drop" si se generaron
df_full = df_full.drop(columns=[col for col in df_full.columns if col.endswith('_drop')])

# 6. Resetear índice
df_full = df_full.reset_index(drop=True)

df_full.drop_duplicates(inplace=True)

df_full.shape

# Detectar si el Equipo es el que inicia el nombre del Partido → Local
df_full['Tipo'] = df_full.apply(
    lambda row: 'Local' if row['Partido'].startswith(row['Equipo']) else 'Visitante',
    axis=1
)


In [56]:
def limpiar_columna(col):
    # Convertir a minúsculas
    col = col.lower()
    # Eliminar tildes
    col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')
    # Reemplazar espacios por guión bajo
    col = col.replace(" ", "_")
    # Quitar caracteres no alfanuméricos ni _
    col = re.sub(r"[^\w_]", "", col)
    # Eliminar guiones bajos al inicio
    col = re.sub(r"^_+", "%", col)
    return col

# Aplicar a todas las columnas
df_full.columns = [limpiar_columna(col) for col in df_full.columns]


In [57]:
ligue = df_full['competicion'].unique()[0]
# df_kk = df_full[df_full['Equipo'] == 'Alianza']

# Ordenar por equipo y fecha
df_full.sort_values(by=["equipo", "fecha"], inplace=True)

# Crear columna 'jornada'
df_full["jornada"] = df_full.groupby("equipo").cumcount() + 1

df_full.to_excel(f"../data/output/{ligue}.xlsx", index=False)
df_full.to_csv(f"../data/output/{ligue}.csv", index=False)
# df_kk[['Fecha', 'Partido', 'Equipo', 'Equipo_contra', 'Tiros','Tiros_contra','xG_contra', 'xG']]

# df_kk['xG_contra'].mean()

In [38]:
# Suma
df_sum_div = df_full.groupby(['equipo', 'tipo'], as_index=False).sum(numeric_only=True)

# Promedio
df_mean_div = df_full.groupby(['equipo', 'tipo'], as_index=False).mean(numeric_only=True)

# Copiar suma
df_per90_div = df_sum_div.copy()

# Columnas a ajustar (todas menos claves)
cols = [col for col in df_per90_div.columns if col not in ['equipo', 'tipo', 'duracion']]

# Duración total por grupo
duracion = df_per90_div['duracion']

# Calcular métricas normalizadas
for col in cols:
    df_per90_div[col] = (90 / duracion) * df_per90_div[col]
    df_per90_div.rename(columns={col: f"{col}_per90"}, inplace=True)

In [39]:
# Agrupaciones base
df_sum = df_full.groupby("equipo", as_index=False).sum(numeric_only=True)
df_mean = df_full.groupby("equipo", as_index=False).mean(numeric_only=True)

# Copiar df_sum para trabajar sobre él
df_per90 = df_sum.copy()

# Guardar y eliminar duración para el cálculo
duracion_total = df_sum['duracion']

# Aplicar fórmula por columna numérica (excepto 'duracion')
cols_to_adjust = [col for col in df_per90.columns if col not in ['equipo', 'duracion']]

for col in cols_to_adjust:
    df_per90[col] = (90 / duracion_total) * df_per90[col]


In [23]:
df_per90[df_per90['equipo']=='Alianza'].head(2)

,equipo,duracion,goles,xg,tiros,tiros_a_la_porteria,%tiros,pases,pases_logrados,%pases,posesion_del_balon_,balones_perdidos,balones_perdidos_bajos,balones_perdidos_medios,balones_perdidos_altos,balones_recuperados,balones_recuperados_bajos,balones_recuperados_medios,balones_recuperados_altos,duelos,duelos_ganados,%duelos,tiros_de_fuera_del_area,tiros_de_fuera_del_area_a_la_porteria,%tiros_de_fuera_del_area,ataques_posicionales,ataques_posicionales_con_remate,%ataques_posicionales,contraataques,contraataques_con_remate,%contraataques,jugadas_a_balon_parado,jugadas_a_balon_parado_con_remate,%jugadas_a_balon_parado,corneres,corneres_con_remate,%corneres,tiros_libres,tiros_libres_con_remate,%tiros_libres,penaltis,penaltis_marcados,%penaltis,centros,centros_precisos,%centros,pases_cruzados_en_profundidad_completados,pases_en_profundidad_completados,entradas_al_area_de_penalti_carreras,entradas_al_area_de_penalti_carreras_pases_cruzados,%entradas_al_area_de_penalti_carreras,toques_en_el_area_de_penalti,duelos_ofensivos,duelos_ofensivos_ganados,%duelos_ofensivos,fuera_de_juego,goles_recibidos,tiros_en_contra,tiros_en_contra_a_la_porteria,%tiros_en_contra,duelos_defensivos,duelos_defensivos_ganados,%duelos_defensivos,duelos_aereos,duelos_aereos_ganados,%duelos_aereos,entradas_a_ras_de_suelo,entradas_a_ras_de_suelo_logradas,%entradas_a_ras_de_suelo,interceptaciones,despejes,faltas,tarjetas_amarillas,tarjetas_rojas,pases_hacia_adelante,pases_hacia_adelante_logrados,%pases_hacia_adelante,pases_hacia_atras,pases_hacia_atras_logrados,%pases_hacia_atras,pases_laterales,pases_laterales_logrados,%pases_laterales,pases_largos,pases_largos_logrados,%pases_largos,pases_en_el_ultimo_tercio,pases_en_el_ultimo_tercio_logrados,%pases_en_el_ultimo_tercio,pases_progresivos,pases_progresivos_precisos,%pases_progresivos,desmarques,desmarques_logrados,%desmarques,saques_laterales,saques_laterales_logrados,%saques_laterales,saques_de_meta,intensidad_de_paso,promedio_pases_por_posesion_del_balon,lanzamiento_largo_,distancia_media_de_tiro,longitud_media_pases,ppda,duracion_contra,goles_contra,xg_contra,tiros_contra,tiros_a_la_porteria_contra,%tiros_contra,pases_contra,pases_logrados_contra,%pases_contra,posesion_del_balon__contra,balones_perdidos_contra,balones_perdidos_bajos_contra,balones_perdidos_medios_contra,balones_perdidos_altos_contra,balones_recuperados_contra,balones_recuperados_bajos_contra,balones_recuperados_medios_contra,balones_recuperados_altos_contra,duelos_contra,duelos_ganados_contra,%duelos_contra,tiros_de_fuera_del_area_contra,tiros_de_fuera_del_area_a_la_porteria_contra,%tiros_de_fuera_del_area_contra,ataques_posicionales_contra,ataques_posicionales_con_remate_contra,%ataques_posicionales_contra,contraataques_contra,contraataques_con_remate_contra,%contraataques_contra,jugadas_a_balon_parado_contra,jugadas_a_balon_parado_con_remate_contra,%jugadas_a_balon_parado_contra,corneres_contra,corneres_con_remate_contra,%corneres_contra,tiros_libres_contra,tiros_libres_con_remate_contra,%tiros_libres_contra,penaltis_contra,penaltis_marcados_contra,%penaltis_contra,centros_contra,centros_precisos_contra,%centros_contra,pases_cruzados_en_profundidad_completados_contra,pases_en_profundidad_completados_contra,entradas_al_area_de_penalti_carreras_contra,entradas_al_area_de_penalti_carreras_pases_cruzados_contra,%entradas_al_area_de_penalti_carreras_contra,toques_en_el_area_de_penalti_contra,duelos_ofensivos_contra,duelos_ofensivos_ganados_contra,%duelos_ofensivos_contra,fuera_de_juego_contra,goles_recibidos_contra,tiros_en_contra_contra,tiros_en_contra_a_la_porteria_contra,%tiros_en_contra_contra,duelos_defensivos_contra,duelos_defensivos_ganados_contra,%duelos_defensivos_contra,duelos_aereos_contra,duelos_aereos_ganados_contra,%duelos_aereos_contra,entradas_a_ras_de_suelo_contra,entradas_a_ras_de_suelo_logradas_contra,%entradas_a_ras_de_suelo_contra,interceptaciones_contra,despejes_contra,faltas_contra,tarjetas_amarillas_contra,tarjetas_rojas_contr

In [40]:
# 1. Filtrar las columnas necesarias
df_xg = df_full[["fecha", "equipo", "xg", "xg_contra"]].copy()

# 2. Ordenar por Equipo y Fecha (ascendente)
df_xg.sort_values(by=["equipo", "fecha"], inplace=True)

# 3. Enumerar por equipo (índice consecutivo por grupo)
df_xg["jornada"] = df_xg.groupby("equipo").cumcount() + 1

# 4. Eliminar la columna Fecha
df_xg.drop(columns=["fecha"], inplace=True)

# 5. (Opcional) Reordenar columnas si deseas
df_xg = df_xg[["equipo", "jornada", "xg", "xg_contra"]]

In [59]:
print(df_full['equipo'].value_counts())

equipo
Chaco For Ever            22
Central Norte             22
Defensores Unidos         22
Temperley                 22
Nueva Chicago             21
Agropecuario              21
Club Atlético Mitre       21
Colón                     21
Defensores de Belgrano    21
Chacarita Juniors         21
Gimnasia Jujuy            21
Estudiantes Río Cuarto    21
Almirante Brown           20
Talleres Remedios         20
Deportivo Morón           19
Estudiantes Caseros       19
Gimnasia Mendoza          19
San Telmo                 19
Name: count, dtype: int64


In [ ]:
Chaco For Ever            22
Central Norte             22
Defensores Unidos         22
Temperley                 22
Nueva Chicago             21
Agropecuario              21
Club Atlético Mitre       21
Colón                     21
Defensores de Belgrano    21
Chacarita Juniors         21
Gimnasia Jujuy            21
Estudiantes Río Cuarto    21
Almirante Brown           20
Talleres Remedios         20
Deportivo Morón           19
Estudiantes Caseros       19
Gimnasia Mendoza          19
San Telmo                 19

## Players


In [43]:
_folder_path_players = '../data/input/LigaEcu/players/'


df_players = pd.read_excel(_folder_path_players + 'players_ecu.xlsx')


In [45]:
df_players

,Jugador,Equipo,Equipo durante el período seleccionado,Posición específica,Edad,Valor de mercado (Transfermarkt),Vencimiento contrato,Partidos jugados,Minutos jugados,Goles,xG,Asistencias,xA,Duelos/90,"Duelos ganados, %",País de nacimiento,Pasaporte,Pie,Altura,Peso,En prestamo,Acciones defensivas realizadas/90,Duelos defensivos/90,"Duelos defensivos ganados, %",Duelos aéreos en los 90,"Duelos aéreos ganados, %",Entradas/90,Posesión conquistada después de una entrada,Tiros interceptados/90,Interceptaciones/90,Posesión conquistada después de una interceptación,Faltas/90,Tarjetas amarillas,Tarjetas amarillas/90,Tarjetas rojas,Tarjetas rojas/90,Acciones de ataque exitosas/90,Goles/90,Goles (excepto los penaltis),"Goles, excepto los penaltis/90",xG/90,Goles de cabeza,Goles de cabeza/90,Remates,Remates/90,"Tiros a la portería, %","Goles hechos, %",Asistencias/90,Centros/90,"Precisión centros, %",Centros desde la banda izquierda/90,"Precisión centros desde la banda izquierda, %",Centros desde la banda derecha/90,"Precisión centros desde la banda derecha, %",Centros al área pequeña/90,Regates/90,"Regates realizados, %",Duelos atacantes/90,"Duelos atacantes ganados, %",Toques en el área de penalti/90,Carreras en progresión/90,Aceleraciones/90,Pases recibidos /90,Pases largos recibidos/90,Faltas recibidas/90,Pases/90,"Precisión pases, %",Pases hacia adelante/90,"Precisión pases hacia adelante, %",Pases hacia atrás/90,"Precision pases hacia atrás, %",Pases laterales/90,"Precisión pases laterales, %",Pases cortos / medios /90,"Precisión pases cortos / medios, %",Pases largos/90,"Precisión pases largos, %","Longitud media pases, m","Longitud media pases largos, m",xA/90,Asistencias/90.1,Second assists/90,Third assists/90,Desmarques/90,"Precisión desmarques, %",Jugadas claves/90,Pases en el último tercio/90,"Precisión pases en el último tercio, %",Pases al área de penalti/90,"Pases hacía el área pequeña, %",Pases en profundidad/90,"Precisión pases en profundidad, %",Ataque en profundidad/90,Centros desde el último tercio/90,Pases progresivos/90,"Precisión pases progresivos, %",Goles recibidos,Goles recibidos/90,Remates en contra,Remates en contra/90,Porterías imbatidas en los 90,"Paradas, %",xG en contra,xG en contra/90,Goles evitados,Goles evitados/90,Pases hacía atrás recibidos del arquero/90,Salidas/90,Duelos aéreos en los 90.1,Tiros libres/90,Tiros libres directos/90,"Tiros libres directos, %",Córneres/90,Penaltis a favor,"Penaltis realizados, %"
0,R. Silva,Orense,Orense,GK,30.0,600000,2026-12-31,33,3361,0,0.00,1,0.14,0.83,80.65,Ecuador,Ecuador,NaN,0,0,no,1.69,0.19,42.86,0.48,88.89,0.03,0.04,0.00,1.58,2.51,0.00,1,0.03,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.00,0.00,0.000,0.03,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.08,100.00,0.00,0.00,0.00,11.14,0.21,0.13,19.98,83.38,12.40,77.32,0.00,0.00,7.07,94.32,11.11,97.59,8.81,65.96,35.17,48.09,0.00,0.03,0.00,0.03,0.00,0.00,0.03,2.09,51.28,0.11,25.00,0.19,42.86,0.03,0.00,6.40,74.48,38,1.02,138,3.7,12,72.46,40.51,1.08,2.51,0.067,0.00,1.85,0.48,NaN,NaN,NaN,0.00,0,0.0
1,I. Zambrano,Libertad,Libertad,LCMF,28.0,150000,NaN,33,3241,4,3.26,1,1.63,13.66,54.67,Ecuador,Ecuador,NaN,0,0,no,6.86,4.83,72.99,2.75,41.41,0.11,0.17,0.33,3.22,4.83,0.75,2,0.06,0,0.0,0.78,0.11,4,0.11,0.09,2,0.06,34,0.94,41.18,11.765,0.03,0.19,28.57,0.08,0.00,0.11,50.0,0.06,0.69,48.00,4.39,45.57,1.89,1.11,0.11,15.22,0.67,1.17,24.21,86.24,6.72,71.90,3.25,94.02,9.64,93.08,22.66,89.09,1.36,46.94,16.45,18.86,0.05,0.58,0.06,0.00,0.08,0.00,0.08,3.08,81.98,0.61,31.82,0.17,16.67,0.50,0.06,3.14,68.14,0,NaN,0,NaN,0,0.00,0.00,0.00,NaN,NaN,0.25,NaN,NaN,0.00,0.00,0.0,0.00,0,0.0
2,J. Cazares,Macará,Macará,"LCMF, RCMF, LDMF",34.0,200000,NaN,34,3214,0,0.82,2,0.99,16.33,51.80,Ecuador,Ecuador,derecho,174,0,no,8.90,7.56,63.70,2.32,44.58,0.67,0.93,0.08,3.42,4.71,1.46,8,0.22,0,0.0,1.09,0.00,0,0.00,0.02,0,0.00,13,0.36,30.77,0.000,0.06,0.56,45.00,0.22,50.00,0.28,40.0,0.00,1.43,49.02,4.45,39.62,0.28,1.20,0.20,22.82,0.42,0.73,36.82,83.35,13.05,70.82,4.09,9